In [1]:
# Import Dependencies
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from glob import glob
from PIL import Image

from keras.preprocessing import image
from keras.datasets import mnist
from keras import optimizers
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, Lambda, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization




In [2]:
# Set up Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Change Directory
%cd '/content/drive/MyDrive/Data_analytics /group_project_4/pet_expression_data_augmented'


/content/drive/MyDrive/Data_analytics /group_project_4/pet_expression_data_augmented


In [3]:
# Define the path to the dataset folders
happy_folder = "/content/drive/MyDrive/Data_analytics /group_project_4/pet_expression_data_augmented/happy"
sad_folder = "/content/drive/MyDrive/Data_analytics /group_project_4/pet_expression_data_augmented/sad"
angry_folder = "/content/drive/MyDrive/Data_analytics /group_project_4/pet_expression_data_augmented/angry"
other_folder = "/content/drive/MyDrive/Data_analytics /group_project_4/pet_expression_data_augmented/other"

In [4]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize to a fixed size for the model
            images.append(img)
    return images

In [5]:
# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)
other_images = load_images_from_folder(other_folder)

In [6]:
# Check shape, 3 means it is in RGB format
happy_images[1].shape

(224, 224, 3)

In [7]:
# Create labels for each emotion category
happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)
other_labels = [3] * len(other_images) # 3???? not sure if thats correct

In [8]:
# Concatenate images and labels
x = np.array(happy_images + sad_images + angry_images + other_images)
y = np.array(happy_labels + sad_labels + angry_labels + other_labels)

In [9]:
# Normalize pixel values to range [0, 1]
x = x.astype('float32') / 255.0

In [10]:
# One-hot encode the labels
y = tf.keras.utils.to_categorical(y, 4)

In [14]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=100)

In [15]:
# Split again into a validation set
# x_validate, x_test, y_validate, y_test= train_test_split (x_test,y_test, test_size=.50, random_state=100)

In [16]:
# Reshape the images to (224, 224, 3)
x_train = np.array([cv2.resize(image, (224, 224)) for image in x_train])
# x_validate = np.array([cv2.resize(image, (224, 224)) for image in x_validate])

In [17]:
print (f' x train',x_train.shape)
print (f' x test', x_test.shape)
print (f' y train', y_train.shape)
print (f' y test', y_test.shape)
# print (f' x validate', x_validate.shape)
# print (f' y validate', y_validate.shape)

 x train (700, 224, 224, 3)
 x test (175, 224, 224, 3)
 y train (700, 4)
 y test (175, 4)


In [18]:
#here [3] denotes for RGB images(3 channels)
# Set the paramaters for the vgg16 model
Image_size = [224, 224]
vgg = VGG16(input_shape= Image_size + [3], weights='imagenet', include_top=False)

#don't train existing weights
for layer in vgg.layers:
 layer.trainable = False

# Set the inputs and outputs for the model
x = Flatten()(vgg.output)
prediction = Dense(4, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

#compile according to common metrics for vgg16 models
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
# Create an ImageDataGenerator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.5
)

# Create a generator for training data
batch_size = 10
train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=batch_size,
    subset='training'  # Specify that this is the training subset
)

# Create a generator for validation data
validation_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=batch_size,
    subset='validation'  # Specify that this is the validation subset
)

# ... (model creation and compilation)

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=5
)

Epoch 1/5
35/35 [==============================] - 286s 8s/step - loss: 0.5392 - acc: 0.2743 - val_loss: 0.5011 - val_acc: 0.3429
Epoch 2/5
35/35 [==============================] - 277s 8s/step - loss: 0.5126 - acc: 0.2800 - val_loss: 0.5016 - val_acc: 0.3429
Epoch 3/5
35/35 [==============================] - 285s 8s/step - loss: 0.5090 - acc: 0.3257 - val_loss: 0.5176 - val_acc: 0.3429
Epoch 4/5
35/35 [==============================] - 282s 8s/step - loss: 0.5119 - acc: 0.3114 - val_loss: 0.5008 - val_acc: 0.3371
Epoch 5/5
35/35 [==============================] - 278s 8s/step - loss: 0.5082 - acc: 0.3343 - val_loss: 0.5118 - val_acc: 0.3371


In [27]:
# #Fit the model
# ###################################################################################
# # Need to check the size of the validation datasets
# # Need to be Make sure that  dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 20 batches)
# history = model.fit(
#                     x_train, y_train,
#                     epochs = 5, verbose=1,
#                     validation_data = (x_validate, y_validate),
#                     validation_steps = 20)

Epoch 1/5
21/21 [==============================] - ETA: 0s - loss: 0.5160 - acc: 0.3841 

21/21 [==============================] - 344s 16s/step - loss: 0.5160 - acc: 0.3841 - val_loss: 0.4442 - val_acc: 0.5138
Epoch 2/5
 9/21 [===========>..................] - ETA: 2:35 - loss: 0.4584 - acc: 0.5312

KeyboardInterrupt: ignored

In [20]:
# Evaluate the model on the train data
train_loss, train_accuracy = model.evaluate(x_train, y_train, verbose=2)
print("Train Accuracy:", train_accuracy)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("Test Accuracy:", test_accuracy)

22/22 - 274s - loss: 0.5225 - acc: 0.3100 - 274s/epoch - 12s/step
Train Accuracy: 0.3100000023841858
6/6 - 68s - loss: 0.5000 - acc: 0.3143 - 68s/epoch - 11s/step
Test Accuracy: 0.3142857253551483


In [ ]:
# Save the trained model
model.save("pet_expressions_model_vgg16.h5")

In [ ]:
##################################################################################

In [ ]:
# Load the saved model

loaded_model = load_model("pet_expressions_model.h5")

In [ ]:
# Load a custom test image

custom_test_image_path = "/kaggle/input/pets-facial-expression-dataset/Angry/16924834.jpg"

custom_test_image = cv2.imread(custom_test_image_path)
custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (48, 48))
custom_test_image = custom_test_image.astype('float32') / 255.0

In [ ]:
# Reshape the image to match the model input shape

custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

In [ ]:
# Make predictions on the custom test image

prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

In [ ]:
emotion_label = np.argmax(prediction[0])

In [ ]:
# Map the predicted label to emotion class

emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry'}
predicted_emotion = emotion_classes[emotion_label]

In [ ]:
# Print the custom test image and its predicted label

print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry]: {prediction_prob}")

In [ ]:
#Display the custom test image using matplotlib

plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# Display the original custom test image using PIL

img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()